In [1]:
# header
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

import shared_utils

import pandas as pd
import geopandas as gpd

pd.set_option('display.max_columns', None) 

import gcsfs

from calitp_data.storage import get_fs
fs = get_fs()

GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/safety_projects/"

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [7]:
# load crashes
switrs = gpd.read_parquet('gs://calitp-analytics-data/data-analyses/project_prioritization/SWITRS_clean.parquet')

In [8]:
switrs_ped = switrs[switrs['pedestrian_accident']=='Y']

In [9]:
# load bridge buffer areas
bridge_areas = gpd.read_parquet(f'{GCS_FILE_PATH}bridgeareas_clean.parquet')

In [10]:
# fixing projection and dropping column - move this over to bridge process script
bridge_areas = bridge_areas.to_crs(3310).drop(["index_right"], axis=1)

In [11]:
# spatial join and aggregate crashes to bridge areas
bridges_crashes = gpd.sjoin(bridge_areas, switrs_ped, how="left")

In [12]:
len(bridges_crashes)

9734

In [13]:
bridges_crashes[bridges_crashes['pedestrian_accident'].notnull()].head(10)

,geometry,BRIDGE_right,OBJECTID_1_left,OBJECTID_left,DIST_left,CO_left,BRIDGE_X_left,BRIDGE_Y_left,LAT_left,LON_left,NAME_left,LOC_left,YRBLT_left,HST_left,FAC_left,APWID_left,LENG_left,DK_AREA_left,LSW_left,RSW_left,RDW_left,REFVCU_left,VCU_left,MAINSPANS_left,DIR_left,PRINC_left,INTERSEC_left,AADT_left,PCTTRK_left,DEF_left,NHS_left,FUNCTIONAL_left,DATA_EXTRA_left,PM_left,CITY_left,RTE_left,OBJECTID_1_right,OBJECTID_right,DIST_right,CO_right,BRIDGE_X_right,BRIDGE_Y_right,LAT_right,LON_right,NAME_right,LOC_right,YRBLT_right,HST_right,FAC_right,APWID_right,LENG_right,DK_AREA_right,LSW_right,RSW_right,RDW_right,REFVCU_right,VCU_right,MAINSPANS_right,DIR_right,PRINC_right,INTERSEC_right,AADT_right,PCTTRK_right,DEF_right,NHS_right,FUNCTIONAL_right,DATA_EXTRA_right,PM_right,CITY_right,RTE_right,index_right,sc_case_id,sc_accident_year,collision_date,collision_time,sc_collision_severity,pcf_viol_category,type_of_collision,mviw,number_killed,number_injured,pedestrian_accident,bicycle_accident,longitude_geo,latitude_geo,collision_date_str,collision_time_str,collision_dt
BRIDGE_left,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
01 0064,"POLYGON ((-348365.690 424943.910, -348365.690 ...",01 0064,29,29,1,DN,0.000000,0,41.772825,-124.186931,WASHINGTON BLVD OC,01-DN-101-R27.87,1971,5,WASHINGTON BLVD,0.0,73.8,1320,0,0,0.0,H,5,2,2,2,U.S. HIGHWAY 101,29500,10,1,1,2,30-Jan-20,R27.87,0,101,29,29,1,DN,0.000000,0,41.772825,-124.186931,WASHINGTON BLVD OC,01-DN-101-R27.87,1971,5,WASHINGTON BLVD,0.0,73.8,1320,0,0,0.0,H,5,2,2,2,U.S. HIGHWAY 101,29500,10,1,1,2,30-Jan-20,R27.87,0,101,74121.0,90568365.0,2017.0,20170920.0,2008.0,1.0,11,G,B,1.0,0.0,Y,None,-124.185648,41.771989,20170920,2008,2017-09-20
01 0064,"POLYGON ((-348365.690 424943.910, -348365.690 ...",01 0064,29,29,1,DN,0.000000,0,41.772825,-124.186931,WASHINGTON BLVD OC,01-DN-101-R27.87,1971,5,WASHINGTON BLVD,0.0,73.8,1320,0,0,0.0,H,5,2,2,2,U.S. HIGHWAY 101,29500,10,1,1,2,30-Jan-20,R27.87,0,101,29,29,1,DN,0.000000,0,41.772825,-124.186931,WASHINGTON BLVD OC,01-DN-101-R27.87,1971,5,WASHINGTON BLVD,0.0,73.8,1320,0,0,0.0,H,5,2,2,2,U.S. HIGHWAY 101,29500,10,1,1,2,30-Jan-20,R27.87,0,101,29658.0,90951556.0,2019.0,20190315.0,1540.0,2.0,3,G,B,0.0,1.0,Y,None,-124.187813,41.772236,20190315,1540,2019-03-15
02 0163L,"POLYGON ((-199324.868 379708.531, -199327.954 ...",02 0163L,154,154,2,SIS,0.000000,0,41.413562,-122.381237,SISKIYOU WAY UC,02-SIS-005-R18.71-WEED,1973,5,INTERSTATE 5 NB,12.2,44.5,589,0,0,12.2,H,5,1,1,1,SISKIYOU WAY,7150,26,1,1,1,30-Jan-20,R18.71,WEED,5,154,154,2,SIS,0.000000,0,41.413562,-122.381237,SISKIYOU WAY UC,02-SIS-005-R18.71-WEED,1973,5,INTERSTATE 5 NB,12.2,44.5,589,0,0,12.2,H,5,1,1,1,SISKIYOU WAY,7150,26,1,1,1,30-Jan-20,R18.71,WEED,5,70531.0,90467108.0,2017.0,20170522.0,2145.0,2.0,11,G,B,0.0,1.0,Y,None,-122.381454,41.413828,20170522,2145,2017-05-22
04 0145,"POLYGON ((-352453.746 294729.437, -352453.746 ...",04 0145,321,321,1,HUM,1.750000,4,40.598894,-124.167267,FORTUNA OH,01-HUM-101-61.53-FTA,1962,5,US HIGHWAY 101,22.6,139.3,3538,1,1,22.6,H,5,6,2,1,"S101-MAIN ON&OFF,NCRRA",12800,11,1,1,12,30-Jan-20,61.53,FORTUNA,101,321,321,1,HUM,1.750000,4,40.598894,-124.167267,FORTUNA OH,01-HUM-101-61.53-FTA,1962,5,US HIGHWAY 101,22.6,139.3,3538,1,1,22.6,H,5,6,2,1,"S101-MAIN ON&OFF,NCRRA",12800,11,1,1,12,30-Jan-20,61.53,FORTUNA,101,38331.0,91154853.0,2019.0,20190827.0,58.0,1.0,11,G,B,1.0,0.0,Y,None,-124.167839,40.599419,20190827,58,2019-08-27
04 0173,"POLYGON ((-346513.645 332556.843, -346513.645 ...",04 0173,335,335,1,HUM,0.000000,0,40.941642,-124.116558,HILLER ROAD OC,01-HUM-101-R91.99,1964,5,HILLER ROAD,9.8,56.7,696,2,0,9.8,H,5,4,2,2,U.S. HIGHWAY 101,12900,11,1,1,12,30-Jan-20,R91.99,0,101,335,335,1,HUM,0.000000,0,40.941642,-124.116558,HILLER ROAD OC,01-HUM-101-R91.99,1964,5,HILLER ROAD,9.8,56.7,696,2,0,9.8,H,5,4,2,2,U.S. HIGHWAY 101,12900,11,1,1,12,30-Jan-20,R91.99,0,101,35843.0,91092609.0,2019.0,20191007.0,1445.0,2.0,8,G,B,0.0,1.0,Y,None,-124.114853,40.94168

In [14]:
# dissolve, grouping by bridge caracteristics, aggregating counts of encampment work orders 
bridges_crashes_agg = bridges_crashes.dissolve(
    by = ["BRIDGE_left","NAME_left","FAC_left"],
    aggfunc = {'number_killed' : 'sum',
               'number_injured' : 'sum',
               'pedestrian_accident' : 'count'
              }
)

In [16]:
bridges_crashes_agg.head(10)

,,,geometry,number_killed,number_injured,pedestrian_accident
BRIDGE_left,NAME_left,FAC_left,,,,
01 0002,MINOT CREEK,U.S. HIGHWAY 101,"POLYGON ((-338420.542 400176.970, -338420.542 ...",0.0,0.0,0
01 0003,HUNTER CREEK,U.S. HIGHWAY 101,"POLYGON ((-338734.315 400686.619, -338734.315 ...",0.0,0.0,0
01 0004,HIGH PRAIRIE CREEK,U.S. HIGHWAY 101,"POLYGON ((-339521.400 401843.252, -339521.400 ...",0.0,0.0,0
01 0005,WILSON CREEK (LOUIS P DEMARTIN SR MEMORIAL BRIDGE),U.S. HIGHWAY 101,"POLYGON ((-342002.868 405967.333, -342002.868 ...",0.0,0.0,0
01 0006,SMITH RIVER (HIOUCHI BRIDGE),U.S. HIGHWAY 199,"POLYGON ((-339518.770 428316.848, -339518.770 ...",0.0,0.0,0
01 0007,MYRTLE CREEK,U.S. HIGHWAY 199,"POLYGON ((-337315.914 427626.518, -337315.914 ...",0.0,0.0,0
01 0009,SMITH RIVER (MARY ADAMS PEACOCK MEMORIAL BRIDGE),U.S. HIGHWAY 199,"POLYGON ((-333477.545 432025.275, -333477.545 ...",0.0,0.0,0
01 0012,MIDDLE FORK SMITH RIVER,U.S. HIGHWAY 199,"POLYGON ((-322941.949 433261.436, -322941.949 ...",0.0,0.0,0
01 0014,PATRICK CREEK,U.S. HIGHWAY 199,"POLYGON ((-319369.416 434997.779, -319369.416 ...",0.0,0.0,0


In [17]:
# fix index
bridges_crashes_agg = bridges_crashes_agg.reset_index()

In [18]:
# save out as geoparquet 
shared_utils.utils.geoparquet_gcs_export(bridges_crashes_agg, GCS_FILE_PATH, "pedcrashes_agg")